# Análisis de ventanas 6-gramas ancladas en fase 2

Este cuaderno replica las dos tareas solicitadas pero en el contexto de ventanas de seis épocas construidas sobre la columna **scor_clean**:

- Se detecta cada transición **no-2 → 2** y se extrae la ventana de seis épocas iniciando en el primer 2.
- Se contabilizan las frecuencias de patrones tipo **2222xx** tanto por paciente como en el conjunto agregado.

Los resultados se presentan únicamente en tablas para que las frecuencias y duraciones sean fáciles de interpretar.

## Plan de trabajo

1. Importar dependencias y definir rutas base.
2. Enumerar pacientes con archivos **Scoring_*_2** y cargar **scor_clean**.
3. **Tarea 1:** generar ventanas 6-gramas por paciente, contar el número de 2 consecutivos al inicio y mostrar tablas de frecuencias por paciente y globales.
4. **Tarea 2:** identificar la fase destino dentro de cada ventana (primer valor distinto de 2), medir cuántas épocas consecutivas ocupa esa fase en la ventana y resumirla en tablas por paciente y globales.
5. Guardar las estructuras intermedias (listas y diccionarios) para futuros cruces con embeddings u otras visualizaciones.


In [18]:
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from IPython.display import display

RUTA_DATOS = Path("../Datos")
ETIQUETAS_FASES = {
    0: "Vigilia (W)",
    1: "N1",
    2: "N2",
    3: "N3",
    4: "N4",
    5: "REM",
    6: "Movimiento",
    7: "Sin clasificar"
}

pd.options.display.float_format = lambda x: f"{x:0.3f}"

## Carga de pacientes y secuencias

In [19]:
def listar_pacientes(ruta_datos=None):
    """Localiza todos los archivos `Scoring_*_2` y devuelve los IDs ordenados."""
    if ruta_datos is None:
        ruta_datos = RUTA_DATOS
    pacientes = []
    for archivo in ruta_datos.iterdir():
        nombre = archivo.name
        if archivo.is_file() and nombre.startswith("Scoring_") and nombre.endswith("_2"):
            pacientes.append(nombre.replace("Scoring_", ""))
    pacientes.sort()
    if not pacientes:
        raise RuntimeError("No se encontraron archivos Scoring_*_2 en la ruta indicada.")
    return pacientes


def cargar_hipnograma(paciente, ruta_datos=None):
    """Carga la columna `scor_clean` completa del paciente y la devuelve como lista de enteros."""
    if ruta_datos is None:
        ruta_datos = RUTA_DATOS
    ruta_archivo = ruta_datos / f"Scoring_{paciente}"
    df = pd.read_csv(ruta_archivo, sep=r"\s+", header=None, names=["epoca", "scor_clean", "scor_quasi"], dtype=int)
    return df["scor_clean"].tolist()


pacientes = listar_pacientes()
print(f"Pacientes detectados ({len(pacientes)}): {', '.join(pacientes)}")


Pacientes detectados (10): AR_2, DG_2, EL_2, GA_2, IN_2, JS_2, LL_2, SS_2, VB_2, VC_2


## Tarea 1 — Ventanas 6-gramas desde cada transición no-2 → 2

Objetivo: medir la estabilidad inicial de la fase 2 dentro de una ventana fija de seis épocas y registrar los patrones completos para detectar secuencias tipo **2222xx**.

Pasos:

1. Detectar cada posición donde aparece **2** precedido por un valor diferente de 2.
2. Extraer la ventana de seis épocas iniciando en esa posición; si no hay seis épocas disponibles al final del registro, se descarta el caso.
3. Contar cuántos valores 2 consecutivos aparecen al inicio de la ventana (prefijo) y guardar el patrón completo.
4. Generar tablas de frecuencias por paciente y una tabla global.

In [20]:
VENTANA = 6

def generar_ventanas_fase2(secuencia, longitud=VENTANA):
    """Extrae todas las ventanas de largo fijo que comienzan cuando hay una transición no-2 → 2."""
    ventanas = []
    prefijos = []
    for indice in range(1, len(secuencia)):
        if secuencia[indice] == 2 and secuencia[indice - 1] != 2:
            cierre = indice + longitud
            if cierre > len(secuencia):
                break  # descartamos la transición si no hay 6 épocas restantes
            ventana = tuple(secuencia[indice:cierre])
            ventanas.append(ventana)
            prefijos.append(contar_prefijo_dos(ventana))
    return ventanas, prefijos


def contar_prefijo_dos(ventana):
    """Cuenta cuántos valores 2 consecutivos aparecen al inicio de la ventana."""
    longitud = 0
    for valor in ventana:
        if valor == 2:
            longitud += 1
        else:
            break
    return longitud


def tabla_prefijos(prefijos):
    """Devuelve la distribución porcentual de longitudes del prefijo N2."""
    if not prefijos:
        return pd.DataFrame(columns=["Prefijo de 2", "Frecuencia", "Porcentaje"])
    conteo = Counter(prefijos)
    total = sum(conteo.values())
    filas = []
    for prefijo in sorted(conteo):
        frecuencia = conteo[prefijo]
        filas.append({
            "Prefijo de 2": prefijo,
            "Frecuencia": frecuencia,
            "Porcentaje": frecuencia / total
        })
    return pd.DataFrame(filas).round(3)


def tabla_patrones(ventanas, limite=20):
    """Lista los patrones 6-grama más frecuentes junto con su proporción relativa."""
    if not ventanas:
        return pd.DataFrame(columns=["Patrón 6-grama", "Frecuencia", "Porcentaje"])
    conteo = Counter("-".join(map(str, v)) for v in ventanas)
    total = sum(conteo.values())
    filas = []
    for patron, frecuencia in conteo.most_common(limite):
        filas.append({
            "Patrón 6-grama": patron,
            "Frecuencia": frecuencia,
            "Porcentaje": frecuencia / total
        })
    return pd.DataFrame(filas)

In [21]:
ventanas_por_paciente = {}  # paciente → lista de ventanas 6-gramas
prefijos_por_paciente = {}  # paciente → lista de longitudes del prefijo N2

for paciente in pacientes:
    hipnograma = cargar_hipnograma(paciente)
    ventanas, prefijos = generar_ventanas_fase2(hipnograma)
    ventanas_por_paciente[paciente] = ventanas
    prefijos_por_paciente[paciente] = prefijos
    print(f"\nPaciente {paciente} — ventanas generadas: {len(ventanas)}")
    display(tabla_prefijos(prefijos).style.format(precision=3).hide(axis="index"))
    print("Patrones más frecuentes (top 20):")
    display(tabla_patrones(ventanas).style.format(precision=3).hide(axis="index"))


Paciente AR_2 — ventanas generadas: 46


Prefijo de 2,Frecuencia,Porcentaje
1,12,0.261
3,1,0.022
4,1,0.022
5,1,0.022
6,31,0.674


Patrones más frecuentes (top 20):


Patrón 6-grama,Frecuencia,Porcentaje
2-2-2-2-2-2,31,0.674
2-3-2-2-2-2,1,0.022
2-2-2-2-2-3,1,0.022
2-3-4-4-4-4,1,0.022
2-3-2-2-2-3,1,0.022
2-2-2-3-2-4,1,0.022
2-4-4-4-3-3,1,0.022
2-3-3-2-3-3,1,0.022
2-3-3-4-4-4,1,0.022
2-1-2-2-2-2,1,0.022



Paciente DG_2 — ventanas generadas: 57


...

Patrón 6-grama,Frecuencia,Porcentaje
2-2-2-2-2-2,17,0.415
2-5-5-5-5-5,5,0.122
2-2-2-0-0-0,2,0.049
2-2-3-2-2-2,2,0.049
2-2-2-2-2-6,2,0.049
2-2-0-1-0-1,1,0.024
2-2-2-3-3-3,1,0.024
2-3-2-3-2-2,1,0.024
2-3-2-2-2-2,1,0.024
2-2-1-1-1-1,1,0.024



Paciente VC_2 — ventanas generadas: 37


Prefijo de 2,Frecuencia,Porcentaje
1,5,0.135
2,6,0.162
3,3,0.081
4,2,0.054
6,21,0.568


Patrones más frecuentes (top 20):


Patrón 6-grama,Frecuencia,Porcentaje
2-2-2-2-2-2,21,0.568
2-5-5-5-5-5,3,0.081
2-2-5-5-5-5,2,0.054
2-2-3-3-4-4,1,0.027
2-2-5-5-5-2,1,0.027
2-2-2-0-0-1,1,0.027
2-2-2-2-6-2,1,0.027
2-2-2-5-5-5,1,0.027
2-2-6-2-2-2,1,0.027
2-2-2-2-3-2,1,0.027


### Cómo interpretar las tablas de la Tarea 1 (ventanas 6-gramas)

- **Prefijo de 2** indica cuántas épocas consecutivas de N2 aparecen antes de que la ventana empiece a mezclar otras fases.
- **Patrón 6-grama** muestra la secuencia completa (seis épocas) más frecuente para cada paciente o en el agregado global.
- Estas tablas permiten localizar ventanas del estilo **2222xx** y estimar qué tan habituales son para cada paciente antes de cruzarlas con embeddings o análisis adicionales.



In [22]:
ventanas_globales = [ventana for lista in ventanas_por_paciente.values() for ventana in lista]
prefijos_globales = [valor for lista in prefijos_por_paciente.values() for valor in lista]

print(f"\nTotal de ventanas generadas en todos los pacientes: {len(ventanas_globales)}")
display(tabla_prefijos(prefijos_globales).style.format(precision=3).hide(axis="index"))
print("Patrones globales más frecuentes (top 20):")
display(tabla_patrones(ventanas_globales).style.format(precision=3).hide(axis="index"))


Total de ventanas generadas en todos los pacientes: 504


Prefijo de 2,Frecuencia,Porcentaje
1,121,0.240
2,59,0.117
3,37,0.073
4,29,0.058
5,30,0.060
6,228,0.452


Patrones globales más frecuentes (top 20):


Patrón 6-grama,Frecuencia,Porcentaje
2-2-2-2-2-2,228,0.452
2-5-5-5-5-5,20,0.040
2-2-2-2-2-6,13,0.026
2-2-2-2-2-3,11,0.022
2-2-3-2-2-2,9,0.018
2-2-2-2-3-2,7,0.014
2-2-2-2-6-2,6,0.012
2-2-5-5-5-5,6,0.012
2-2-2-5-5-5,5,0.010
2-2-2-3-2-3,5,0.010


## Tarea 2 — Fase destino y duración dentro de la ventana

Ahora se analiza la porción de la ventana en la que la secuencia abandona la fase 2. Para cada 6-grama:

1. Se localiza el primer valor distinto de 2 (fase destino) dentro de la ventana. Si toda la ventana es 2, se ignora porque no hubo transición visible.
2. Se cuenta cuántas épocas consecutivas ocupa la fase destino antes de aparecer otro valor diferente dentro de la misma ventana.
3. Esa duración queda asociada a la fase destino correspondiente.

Las tablas resultantes muestran cuántas veces aparece cada destino **2 → X** y cuánto dura, en promedio, esa fase dentro de la ventana de seis épocas (limitada al contexto observado).

In [23]:
def analizar_destinos_en_ventanas(ventanas):
    """Compila la duración de cada fase destino visible dentro de las ventanas de 6 épocas."""
    destinos = defaultdict(list)
    for ventana in ventanas:
        destino, duracion = destino_y_duracion(ventana)
        if destino is not None:
            destinos[destino].append(duracion)
    return destinos


def destino_y_duracion(ventana):
    """Devuelve la fase destino y cuántas épocas consecutivas ocupa dentro de la ventana."""
    for indice, valor in enumerate(ventana):
        if valor != 2:
            duracion = 1
            cursor = indice + 1
            while cursor < len(ventana) and ventana[cursor] == valor:
                duracion += 1
                cursor += 1
            return valor, duracion
    return None, None


def tabla_destinos(destinos):
    """Genera una tabla resumen con las métricas básicas por transición 2 → X."""
    filas = []
    for fase_destino in sorted(destinos):
        longitudes = destinos[fase_destino]
        if not longitudes:
            continue
        serie = pd.Series(longitudes)
        filas.append({
            "Fase destino": fase_destino,
            "Etiqueta": ETIQUETAS_FASES.get(fase_destino, str(fase_destino)),
            "Segmentos": len(longitudes),
            "Promedio": serie.mean(),
            "Mínimo": serie.min(),
            "Máximo": serie.max()
        })
    columnas = ["Fase destino", "Etiqueta", "Segmentos", "Promedio", "Mínimo", "Máximo"]
    return pd.DataFrame(filas, columns=columnas).round(3)

In [24]:
destinos_por_paciente = {}  # paciente → {fase destino: lista de duraciones}

for paciente, ventanas in ventanas_por_paciente.items():

    destinos = analizar_destinos_en_ventanas(ventanas)
    destinos_por_paciente[paciente] = destinos
    total_transiciones = sum(len(v) for v in destinos.values())
    print(f"\nPaciente {paciente} — transiciones 2 → X detectadas en la ventana: {total_transiciones}")
    display(tabla_destinos(destinos).style.format(precision=3).hide(axis="index"))


Paciente AR_2 — transiciones 2 → X detectadas en la ventana: 15


Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
1,N1,1,1.000,1,1
3,N3,11,1.727,1,5
4,N4,1,3.000,3,3
5,REM,1,5.000,5,5
6,Movimiento,1,1.000,1,1



Paciente DG_2 — transiciones 2 → X detectadas en la ventana: 36


Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
0,Vigilia (W),2,1.000,1,1
1,N1,6,1.167,1,2
3,N3,14,1.429,1,3
5,REM,6,3.167,1,5
6,Movimiento,8,1.625,1,3



Paciente EL_2 — transiciones 2 → X detectadas en la ventana: 32


...

Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
0,Vigilia (W),1,2.000,2,2
1,N1,1,1.000,1,1
3,N3,15,2.333,1,5
4,N4,1,4.000,4,4
5,REM,1,5.000,5,5
6,Movimiento,7,1.143,1,2



Paciente VB_2 — transiciones 2 → X detectadas en la ventana: 24


Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
0,Vigilia (W),6,2.167,1,4
1,N1,2,3.000,2,4
3,N3,7,1.429,1,3
5,REM,7,4.286,1,5
6,Movimiento,2,1.000,1,1



Paciente VC_2 — transiciones 2 → X detectadas en la ventana: 16


Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
0,Vigilia (W),1,2.000,2,2
3,N3,6,1.500,1,2
5,REM,7,4.143,3,5
6,Movimiento,2,1.000,1,1


### Cómo leer las tablas de la Tarea 2 (ventanas 6-gramas)

- **Fase destino / Etiqueta** señala la primera fase distinta de N2 que aparece dentro de la ventana.
- **Segmentos** cuenta cuántas ventanas mostraron cada transición **2 → X** con suficiente contexto para medir cuánto dura X.
- **Promedio / Mínimo / Máximo** indican cuántas épocas consecutivas se observó la fase destino dentro de la misma ventana (máximo 5 épocas porque la ventana tiene tamaño fijo). Estas cifras ayudan a detectar ventanas donde, por ejemplo, **2 → 5** ocupa gran parte del 6-grama y podría ser prioritaria para analizar vecinos en el espacio de embeddings.



In [25]:
destinos_globales = defaultdict(list)
for destinos in destinos_por_paciente.values():
    for fase_destino, longitudes in destinos.items():
        destinos_globales[fase_destino].extend(longitudes)

total_global = sum(len(v) for v in destinos_globales.values())
print(f"\nTransiciones 2 → X detectadas en la ventana (todos los pacientes): {total_global}")
display(tabla_destinos(destinos_globales).style.format(precision=3).hide(axis="index"))


Transiciones 2 → X detectadas en la ventana (todos los pacientes): 276


Fase destino,Etiqueta,Segmentos,Promedio,Mínimo,Máximo
0,Vigilia (W),16,1.875,1,4
1,N1,21,1.476,1,5
3,N3,132,1.606,1,5
4,N4,11,2.818,1,5
5,REM,42,3.762,1,5
6,Movimiento,53,1.208,1,3
7,Sin clasificar,1,1.000,1,1


## Datos listos para cruces posteriores

- **ventanas_por_paciente**: diccionario con todas las ventanas 6-gramas detectadas en cada paciente.
- **prefijos_por_paciente**: longitudes del tramo inicial de 2 en cada ventana.
- **ventanas_globales** y **prefijos_globales**: agregados útiles para revisar los patrones más frecuentes en el total de pacientes.
- **destinos_por_paciente** y **destinos_globales**: duraciones observadas de cada fase destino dentro de la ventana de seis épocas.

Estas estructuras permiten seleccionar patrones específicos (por ejemplo **2222xx**) para cruzarlos con embeddings u otros análisis sin volver a procesar los hipnogramas.